In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats, signal
import numpy as np
from sklearn import preprocessing
from sklearn.linear_model import LinearRegression, HuberRegressor

In [ ]:
df = pd.read_csv('skab.csv', sep=";", index_col=0, parse_dates=True)

In [ ]:
df.describe()

In [ ]:
df.isna().any()

In [ ]:
df.columns

In [ ]:
plt.imshow(df.corr(), cmap='seismic')
plt.colorbar()
plt.gca().set_xticks(np.arange(len(df.columns)))
plt.gca().set_yticks(np.arange(len(df.columns)))
plt.gca().set_xticklabels(labels=df.columns)
plt.gca().set_yticklabels(labels=df.columns)
plt.setp(plt.gca().get_xticklabels(), rotation=45, ha="right",
         rotation_mode="anchor")
plt.tight_layout()
plt.show()

In [ ]:
fig, axs = plt.subplots(nrows=1, ncols=len(df.columns), figsize=(35, 6))

for i, col in enumerate(df):
    axs[i].plot(df[col])
    axs[i].set_title(col)
plt.tight_layout()
plt.show()

### Outliers por IQR

In [ ]:
def filter_outliers(column):
    column_iqr = stats.iqr(column)
    lower_bound = np.percentile(column, 25) - 1.5 * column_iqr
    upper_bound = np.percentile(column, 75) + 1.5 * column_iqr
    return column[(column >= lower_bound) & (column <= upper_bound)]


df_iqr = df.apply(filter_outliers, axis=0).dropna()

In [ ]:
fig, axs = plt.subplots(nrows=1, ncols=len(df.columns), figsize=(35, 6))
for i, col in enumerate(df):
    axs[i].plot(df[col])
    axs[i].set_title(col)
plt.tight_layout()
plt.show()

### Outliers por hampel

peguei de https://github.com/dwervin/pyhampel

tem tambem o pacote hampel pra ser instalado via pip, mas precisa instalar o C++ compiler e eu não tava afim

In [ ]:
def hampel_filter_df(df: pd.DataFrame, vals_col: str, time_col=None, win_size=30, num_dev=3, center_win=True) -> pd.DataFrame:
    """
    This function takes in dataframe containing time series of values, applies Hampel filter on
    these values, and returns dataframe consisting of original values columns along with
    the Hampel filtered data, outlier values and boolean flags where outliers found.

    Parameters
    ----------
    df: pd.DataFrame
        data from containing time series that needs to be Hampel filtered
    vals_col: str
        Single column name that contains values that need to be filtered.
    time_col: str
        Name of column that contains dates or timestamps
    win_size: int
        Size of sliding window for filtering.  Essentially the number of time steps to be considered when filtering.
    num_dev: int
        Number of standard deviations to consider when detecting values that would be considered outliers.
    center_win: Boolean
        Boolean value that determines whether the window is centered about the point being filtered?  Default=True.
        If False, point is at the leading edge (i.e. right side) of window  calculation.

    Returns
    -------
    Function returns a full dataframe consisting of original values columns along with
    the Hampel filtered data, outlier values and boolean flags where outliers found.
    """
    # print("IN HAMPEL_FILTER_DF")

    if (time_col != None):
        if (time_col not in list(df.columns)):
            raise Exception("Timestamp column '{}' is missing!".format(time_col))
        elif (time_col in list(df.columns)):
            if (not np.issubdtype(df[time_col].dtype, np.datetime64)):
                if (not np.issubdtype(pd.to_datetime(df[time_col]).dtype, np.datetime64)):
                    raise Exception("Timestamp column '{}' is not np.datetime64".format(time_col))
                else:
                    df[time_col] = pd.to_datetime(df[time_col])
                    drop_cols = set(df.columns) - set([time_col, vals_col])
                    # Not really filtered at this point. Just naming appropriately ahead of time.
                    orig_vals = df.sort_values(time_col, ascending=True).set_index(time_col).copy()
                    filtered = orig_vals.drop(columns=drop_cols).copy()
            else:
                df[time_col] = pd.to_datetime(df[time_col])
                drop_cols = set(df.columns) - set([time_col, vals_col])
                # Not really filtered at this point. Just naming appropriately ahead of time.
                orig_vals = df.sort_values(time_col, ascending=True).set_index(time_col).copy()
                filtered = orig_vals.drop(columns=drop_cols).copy()

    elif (time_col == None):
        if (not isinstance(df.index, pd.DatetimeIndex)):
            raise Exception("DataFrame index is not pd.DatetimeIndex")
        else:
            df.sort_index(inplace=True)
            drop_cols = set(df.columns) - set([vals_col])
            orig_vals = df.copy()
            filtered = orig_vals.drop(columns=drop_cols).copy()

    # Scale factor for estimating standard deviation based upon median value
    L = 1.4826

    # Calculate rolling median for the series
    rolling_median = filtered.rolling(window=int(win_size), center=center_win, min_periods=1).median()

    # Define a lambda function to apply to the series to calculate Median Absolute Deviation
    def MAD(x): return np.median(np.abs(x - np.median(x)))

    # Calculate rolling MAD series
    rolling_MAD = filtered.rolling(window=(win_size), center=center_win, min_periods=1).apply(MAD)

    # Calculate threshold level for filtering based upon the number of standard deviation and
    # constant scaling factor L.
    threshold = int(num_dev) * L * rolling_MAD

    # Difference between original values and rolling median
    # Again, "filtered" not yet filtered at this point.
    difference = np.abs(filtered - rolling_median)

    '''
    # TODO: Look at logic here to possibly not mark as an outlier if threshold value
    is 0.0
    '''

    # Flag outliers
    outlier_idx = difference > threshold

    # Now it's filtered.  This should replace original values with filtered values from the rolling_median
    # dataframe where outliers were found.
    filtered[outlier_idx] = rolling_median[outlier_idx]
    filtered.rename(columns={vals_col: 'FLTRD_VAL'}, inplace=True)

    # Capture outliers column
    outliers = orig_vals[outlier_idx].rename(columns={vals_col: 'OUTLIER_VAL'}).drop(columns=drop_cols)
    # Capture outlier IS_OUTLIER column
    outlier_idx.rename(columns={vals_col: 'IS_OUTLIER'}, inplace=True)

    # The following returns a full dataframe consisting of original values columns
    # along with the Hampel filtered data, outlier values and boolean flags where outliers found.
    return outlier_idx

In [ ]:
df_hampel = df.copy()
for col in df.columns:
    outliers = hampel_filter_df(df=df_hampel, vals_col=col, win_size=20, num_dev=3, center_win=True)
    try:
        df_hampel = df_hampel[~outliers.values]
    except:
        pass

In [ ]:
fig, axs = plt.subplots(nrows=1, ncols=len(df_hampel.columns), figsize=(35, 6))
for i, col in enumerate(df_hampel):
    axs[i].plot(df_hampel[col])
    axs[i].set_title(col)
plt.tight_layout()
plt.show()

### Filtro Savitzky-Golay

In [ ]:
filtered_df = df_iqr.copy()
for col in filtered_df.columns:
    filtered_df[col] = signal.savgol_filter(df_iqr[col], 15, 3)

In [ ]:
fig, axs = plt.subplots(nrows=1, ncols=len(filtered_df.columns), figsize=(35, 6))
for i, col in enumerate(filtered_df):
    axs[i].plot(filtered_df[col])
    axs[i].set_title(col)
plt.tight_layout()
plt.show()

### Escalonamento com MinMaxScaler

In [ ]:
df_min_max = filtered_df.copy()
min_max_scaler = preprocessing.MinMaxScaler()
df_min_max[df_min_max.columns] = min_max_scaler.fit_transform(filtered_df)
df_min_max

### Escalonamento com StandardScaler

In [ ]:
df_standard = filtered_df.copy()
standard_scaler = preprocessing.StandardScaler()
df_standard[df_standard.columns] = standard_scaler.fit_transform(filtered_df)
df_standard

### E.E. determinando slices

In [ ]:
n_diff_window = 1600
var = 'Temperature'
EE_tol_coef = 5e-5

#######################################
ee_df = df_iqr[var].copy().to_frame().reset_index(drop=True)
ee_df[var] = signal.savgol_filter(ee_df[var], 200, 1)
plt.plot(ee_df)
plt.tight_layout()
plt.show()
split_list = [ee_df[i:i+n_diff_window] for i in range(0, ee_df.shape[0], n_diff_window)]
model = LinearRegression()
coefs = np.empty(len(split_list))
for i, split in enumerate(split_list):
    X = np.array(split.index)
    y = split.values
    model.fit(X.reshape(-1, 1), y.ravel())
    coef_ = model.coef_ * X + model.intercept_
    plt.plot(X, coef_)
    coefs[i] = model.coef_[0]
plt.show()
plt.scatter(range(len(coefs)), coefs)
plt.show()
ee_true_point = np.where(coefs == 0)[0].round(5)  # se deixar livre, nunca vai ter um true EE
if len(ee_true_point) > 0:
    ee_index = split_list[ee_true_point[0]].index[0]
    print("True EE index:", ee_index)
elif any(np.abs(coefs) < EE_tol_coef):
    ee_index = split_list[np.argmin(np.abs(coefs))].index[0]
    print("Aproximado EE start index:", ee_index)
else:
    print("Nenhum EE identificado:")

if ee_index:
    plt.plot(ee_df)
    plt.vlines(ee_index, min(ee_df[var]), max(ee_df[var]), color='red')
    plt.show()

### E.E. com uma janela movel, sem determinar slices

In [ ]:
rolling_window_size = 1000
rolling_speed = 500
var = 'Temperature'
EE_tol_coef = 5e-5

#######
ee_df = df_iqr[var].copy().to_frame().reset_index(drop=True)
ee_df[var] = signal.savgol_filter(ee_df[var], 200, 1)
plt.plot(ee_df)
plt.tight_layout()
plt.show()
model = LinearRegression()
coefs = []
lines = []
for i in range(0, len(ee_df), rolling_speed):
    X = np.array(ee_df.index[i:i+rolling_window_size])
    y = ee_df.iloc[i:i+rolling_window_size].values
    model.fit(X.reshape(-1, 1), y.ravel())
    coef_ = model.coef_ * X + model.intercept_
    lines.append((X, coef_))
    coefs.append(model.coef_[0])
coefs = np.array(coefs)
plt.figure()
for line in lines:
    plt.plot(line[0], line[1])
plt.show()

plt.scatter(range(len(coefs)), coefs)
plt.show()

ee_true_point = np.where(coefs == 0)[0].round(5)  # se deixar livre, nunca vai ter um true EE
if len(ee_true_point) > 0:
    ee_index = lines[ee_true_point][0][0]
    print("True EE index:", ee_index)
elif any(np.abs(coefs) < EE_tol_coef):
    ee_index = lines[np.argmin(np.abs(coefs))][0][0]
    print("Aproximado EE start index:", ee_index)
else:
    print("Nenhum EE identificado:")

if ee_index:
    plt.plot(ee_df)
    plt.vlines(ee_index, min(ee_df[var]), max(ee_df[var]), color='red')
    plt.show()